In [50]:
from pathlib import Path
import numpy as np

from neurodsp import fourier, utils
from scipy import signal

import spikeinterface as si

import one.alf.io as alfio

from tqdm import tqdm
import glob
import os

In [51]:
probe_name = 'ProbeB'

data_directories = glob.glob('/data/ecephys_*')
data_directories.sort()

session_folder = Path(data_directories[0])
sorting_folder = Path(data_directories[1])
scratch_folder = Path('/scratch')
results_folder = Path('/results')
output_folder = results_folder / probe_name

if not os.path.exists(output_folder):
    os.mkdir(output_folder)
ecephys_compressed_folder = session_folder / 'ecephys_compressed'
sorting_curated_folder = sorting_folder / "sorting_precurated"
postprocessed_folder = sorting_folder / 'postprocessed'
ap_stream_name = f'experiment1_Record Node 104#Neuropix-PXI-100.{probe_name}-AP'
lfp_stream_name = f'experiment1_Record Node 104#Neuropix-PXI-100.{probe_name}-LFP'

In [52]:
we_recless = si.load_waveforms(postprocessed_folder / f'{ap_stream_name}_recording1', with_recording=False)

channel_inds = np.array([int(name[2:])-1 for name in we_recless.channel_ids])

In [53]:
RMS_WIN_LENGTH_SECS = 3
WELCH_WIN_LENGTH_SAMPLES = 1024

In [54]:
for is_lfp in (False, True):

    if is_lfp:
        stream_name = lfp_stream_name
    else:
        stream_name = ap_stream_name
    
    recording = si.read_zarr(ecephys_compressed_folder / f"{stream_name}.zarr")

    print(recording.sampling_frequency)

    rms_win_length_samples = 2 ** np.ceil(np.log2(recording.sampling_frequency * RMS_WIN_LENGTH_SECS))

    # the window generator will generates window indices
    wingen = utils.WindowGenerator(ns=recording.get_num_samples(), nswin=rms_win_length_samples, overlap=0)

    win = {'TRMS': np.zeros((wingen.nwin, recording.get_num_channels())),
           'nsamples': np.zeros((wingen.nwin,)),
           'fscale': fourier.fscale(WELCH_WIN_LENGTH_SAMPLES, 1 / recording.sampling_frequency, one_sided=True),
           'tscale': wingen.tscale(fs=recording.sampling_frequency)}
    
    win['spectral_density'] = np.zeros((len(win['fscale']), recording.get_num_channels()))

    with tqdm(total=wingen.nwin) as pbar:
        for first, last in wingen.firstlast:
            D = recording.get_traces(start_frame=first, end_frame=last).T
            # remove low frequency noise below 1 Hz
            D = fourier.hp(D, 1 / recording.sampling_frequency, [0, 1])
            iw = wingen.iw
            win['TRMS'][iw, :] = utils.rms(D)
            win['nsamples'][iw] = D.shape[1]
            # the last window may be smaller than what is needed for welch
            if last - first < WELCH_WIN_LENGTH_SAMPLES:
                continue
            # compute a smoothed spectrum using welch method
            _, w = signal.welch(
                D, fs=recording.sampling_frequency, window='hann', nperseg=WELCH_WIN_LENGTH_SAMPLES,
                detrend='constant', return_onesided=True, scaling='density', axis=-1
            )
            win['spectral_density'] += w.T
            # print at least every 20 windows
            if (iw % min(20, max(int(np.floor(wingen.nwin / 75)), 1))) == 0:
                pbar.update(iw)
                
    win['TRMS'] = win['TRMS'][:,channel_inds]
    win['spectral_density'] = win['spectral_density'][:,channel_inds]

    if is_lfp:
        alf_object_time = f'ephysTimeRmsLF'
        alf_object_freq = f'ephysSpectralDensityLF'
    else:
        alf_object_time = f'ephysTimeRmsAP'
        alf_object_freq = f'ephysSpectralDensityAP'
    
    tdict = {'rms': win['TRMS'].astype(np.single), 'timestamps': win['tscale'].astype(np.single)}
    alfio.save_object_npy(output_folder, object=alf_object_time, dico=tdict, namespace='iblqc')
    
    fdict = {'power': win['spectral_density'].astype(np.single),
             'freqs': win['fscale'].astype(np.single)}
    alfio.save_object_npy(
        output_folder, object=alf_object_freq, dico=fdict, namespace='iblqc')

30000.0


54150it [4:24:51,  3.41it/s]                        


2500.0


91200it [36:05, 42.12it/s]                         
